In [1]:
from selenium import webdriver
# By : Tag에 접근하기 편하게 하는 기능
from selenium.webdriver.common.by import By
# Keys : 키보드의 이벤트
from selenium.webdriver.common.keys import Keys
import pandas as pd 
from sqlalchemy import create_engine

In [5]:
# 웹드라이버를 이용하여 구글크롬 브라우져를 오픈 
driver =webdriver.Chrome()

In [6]:
# 특정 주소로 요청을 보낸다. 
driver.get('http://www.naver.com')

In [7]:
# Tag 중 id가 query인 태그를 선택하여 변수에 저장 
search_element = \
    driver.find_element(By.ID, 'query')

In [9]:
type(search_element)

selenium.webdriver.remote.webelement.WebElement

In [10]:
# search_element(검색어창)에 특정 텍스트를 
# 입력(데이터를 보낸다)한다. 
search_element.send_keys('아이폰')

In [11]:
# search_element에서 ENTER 이벤트를 발생
search_element.send_keys(Keys.ENTER)

In [13]:
# 쇼핑 버튼(Tag)을 선택한다. 
# LINK_TEXT가 쇼핑인 태그 -> 
# 해당 조건에 맞는 태그가 몇개인가?
len(driver.find_elements(By.LINK_TEXT, '쇼핑'))
# 조건에 맞는 태그의 개수가 1개 이므로 클릭 이벤트 발생
driver.find_element(By.LINK_TEXT, '쇼핑').click()

In [14]:
# 쇼핑 페이지에서 검색어의 결과(물건의 가격과 제품명)를 
# 크롤링 -> 해당 페이지의 소스코드를 불러온다. 
html_data = driver.page_source

In [15]:
# bs4 라이브러리 안에 있는 BeautifulSoup을 이용하여 
# html_data를 parsing(데이터의 타입을 변환)
from bs4 import BeautifulSoup as bs

In [16]:
soup = bs(html_data, 'html.parser')